# LLM: Zero-shot classification through LLMs and prompts

Classify data into categories without being explicitly trained on labeled examples for that specific task. Instead, the model uses pre-trained knowledge and natural language prompts to infer the correct category.

| Prompt Name              | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|--------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **simple_prompt**        | "Respond with YES or NO. Does the following person develop a psychological disorder between time point T1 and T2? Data: …"                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| **class_definitions_prompt** | "Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Data: …" |
| **profiled_simple_prompt** | "You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Tell me if the following data shows the development of a psychological disorder between time point T1 and T2. Just answer YES or NO. Data: …"                                                                                                                                                  |
| **few_shot_prompt**      | "Select the top-5 semantically similar examples to the test product from the training set, using the Sentence-Transformer model, examples are then included in the prompt as in-context learning examples for the LLMs. Prompt 1: Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: Example 1: … Prompt 2: Based on the previous example data prompts, classify the following data. Does the following person develop a psychological disorder between time point T1 and T2? Data: …"                                                                       |
| **vignette_text_prompt** | "Data is in csv format, but what if we prompt ChatGPT to convert the table format into a vivid/interesting/emotional text? → vignette. Vignette: Each fictional patient was assigned an age, sex, physical examination findings, and clinical symptoms. ‘Can you grade this case according to … Classification System?’"                                                                                                                                                                                                                                                                           |

## 0 Imports

In [103]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist

In [84]:
data = pd.read_csv("../dat/dips/DIPS_Data_cleaned.csv", sep = ",", low_memory = False)
data_pred = pd.read_csv("../dat/dips/DIPS_Data_cleaned_pred.csv", sep = ",", low_memory = False)
data_pred_y = pd.read_csv("../dat/dips/DIPS_Data_cleaned_pred_y.csv", sep = ",", low_memory = False)

## 1 Zero-shot classification

### 1.1 Data preparation

In [96]:
# Predictors
X = data
X = X.drop(["hpi"], axis = 1)

# Target
y = data["hpi"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# concatenate X and y for example data
dat_train = pd.concat([X_train, y_train], axis = 1)

print("LLMs \n",
      "X_train shape: ", X_train.shape, round(X_train.shape[0]/len(X), 2), "\n",
      "X_test shape: ", X_test.shape, round(X_test.shape[0]/len(X), 2),  "\n",
      "y_train shape: ", y_train.shape, round(y_train.shape[0]/len(y), 2), "\n",
      "y_test shape: ", y_test.shape, round(y_test.shape[0]/len(y), 2), "\n")

LLMs 
 X_train shape:  (1005, 22) 0.8 
 X_test shape:  (252, 22) 0.2 
 y_train shape:  (1005,) 0.8 
 y_test shape:  (252,) 0.2 



In [66]:
# get clear names for each column in data
preds = X.columns
preds

Index(['whi_ges', 'soz_ges', 'gke_ges', 'lzh_ges', 'woc_gesp', 'woc_gese',
       'asi_ges', 'bsq_ges', 'das_ges', 'scl_gsi', 'bild', 'bmi_kat',
       'ses_kom', 'whi_gesy', 'soz_gesy', 'gke_gesy', 'lzh_gesy', 'asi_gesy',
       'bsq_gesy', 'das_gesy', 'scl_gsiy', 'ile'],
      dtype='object')

In [67]:
predictor_names = ["T1 Positive mental health",    #whi_ges
                   "T1 Social support",            #soz_ges (SAVE)
                   "T1 General self-efficacy",     #gke_ges SAVE
                   "T1 Life satisfaction",         #lzh_ges SAVE
                   "T1 Problem-focused coping",    #woc_gesp SAVE
                   "T1 Emotion-focused coping",    #woc_gese SAVE
                   "T1 Anxiety sensitivity",       #asi_ges SAVE
                   "T1 Fear of bodily sensations", #bsq_ges SAVE
                   "T1 Dysfunctional attitudes",   #das_ges SAVE
                   "T1 General psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
                   "Education",                    #bild
                   "T1 BMI",                          #bmi_kat
                   "Socioeconomic status",         #ses_kom
                   "T2 Positive mental health",    #whi_gesy
                   "T2 Social support",            #soz_gesy (SAVE)
                   "T2 General self-efficacy",     #gke_gesy SAVE
                   "T2 Life satisfaction",         #lzh_gesy SAVE
                   "T2 Anxiety sensitivity",       #asi_gesy SAVE
                   "T2 Fear of bodily sensations", #bsq_gesy SAVE
                   "T2 Dysfunctional attitudes",   #das_gesy SAVE
                   "T2 General psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
                   "T1 Stress"                     #ile SAVE
                   ]

target_names = ["Total mental disorders incidence point prevalence"]    #hpi

In [68]:
# print col and names
for col, name in zip(preds, predictor_names):
    print(f"{col}: {name}")

whi_ges: T1 Positive mental health
soz_ges: T1 Social support
gke_ges: T1 General self-efficacy
lzh_ges: T1 Life satisfaction
woc_gesp: T1 Problem-focused coping
woc_gese: T1 Emotion-focused coping
asi_ges: T1 Anxiety sensitivity
bsq_ges: T1 Fear of bodily sensations
das_ges: T1 Dysfunctional attitudes
scl_gsi: T1 General psychopathology: Global Severity Index (GSI)
bild: Education
bmi_kat: T1 BMI
ses_kom: Socioeconomic status
whi_gesy: T2 Positive mental health
soz_gesy: T2 Social support
gke_gesy: T2 General self-efficacy
lzh_gesy: T2 Life satisfaction
asi_gesy: T2 Anxiety sensitivity
bsq_gesy: T2 Fear of bodily sensations
das_gesy: T2 Dysfunctional attitudes
scl_gsiy: T2 General psychopathology: Global Severity Index (GSI)
ile: T1 Stress


#### 1.1 Testing prompting

In [69]:
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # testing
# response = client.responses.create(
#     model = "gpt-4o",
#     instructions = "You are a coding assistant that talks like a pirate.",
#     input = "How do I check if a Python object is an instance of a class?",
# )
#
# print(response.output_text)

### 1.2 Simple prompt

In [70]:
def create_simple_prompt(data_row):
    """
    Create a simple prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond with YES or NO."

    prompt = f"Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [87]:
# Test the function
data_row = X.iloc[4]
simple_instrcution, simple_prompt = create_simple_prompt(data_row)
print(simple_prompt)
print("\n", target_names[0], y.iloc[4])

Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.0346394734930797, T1 Social support: -1.9443495599450165, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -1.2777276005686404, T1 Problem-focused coping: -0.6434380872019897, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: -1.6912121763530454, T1 Fear of bodily sensations: -1.3399218609334862, T1 Dysfunctional attitudes: 1.528535706802646, T1 General psychopathology: Global Severity Index (GSI): -0.5273858134379611, Education: 1.0, T1 BMI: 1.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: 1.0533586187543231, T2 General self-efficacy: 1.3559331135757966, T2 Life satisfaction: 1.005628346789385, T2 Anxiety sensitivity: -1.1666271173074825, T2 Fear of bodily sensations: -1.0416157340293282, T2 Dysfunctional attitudes: -0.8267579141727915, T2 General psychopathology: Global S

In [72]:
# Create a simple prompt for each row in the test set
X_test_simple_prompt = []

for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    simple_instrcution, simple_prompt = create_simple_prompt(data_row)
    X_test_simple_prompt.append(simple_prompt)

X_test_simple_prompt[0]

'Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9916025104337652, T1 Social support: 0.1912554943776842, T1 General self-efficacy: -1.209478876081861, T1 Life satisfaction: 0.1815598805259552, T1 Problem-focused coping: -1.0834346466693496, T1 Emotion-focused coping: -0.1263180001181833, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sensations: 1.2360549695511902, T1 Dysfunctional attitudes: -0.6885415900752596, T1 General psychopathology: Global Severity Index (GSI): -0.4917971179478549, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.7828728972526869, T2 Social support: 0.0984652458135236, T2 General self-efficacy: -1.1421997527016898, T2 Life satisfaction: -0.6556858132553557, T2 Anxiety sensitivity: 1.5262249302236135, T2 Fear of bodily sensations: 1.9574857842600144, T2 Dysfunctional attitudes: -0.2807869674967928, T2 General psychopathology: Global Se

In [73]:
# create random input for testing
data_row = np.random.rand(len(predictor_names))
simple_prompt = create_simple_prompt(data_row)
print(simple_prompt)

('Respond with YES or NO.', 'Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: 0.5778876062325962, T1 Social support: 0.9450764488704365, T1 General self-efficacy: 0.9085259159932666, T1 Life satisfaction: 0.4629799735146718, T1 Problem-focused coping: 0.4226543683134041, T1 Emotion-focused coping: 0.8404291445653612, T1 Anxiety sensitivity: 0.5242714848188041, T1 Fear of bodily sensations: 0.3484341338087932, T1 Dysfunctional attitudes: 0.796467444058449, T1 General psychopathology: Global Severity Index (GSI): 0.18410668150915444, Education: 0.03516147431273742, T1 BMI: 0.606873695545266, Socioeconomic status: 0.6776975324065688, T2 Positive mental health: 0.08692069393377055, T2 Social support: 0.07987535376885335, T2 General self-efficacy: 0.9460360476499977, T2 Life satisfaction: 0.5591940994036095, T2 Anxiety sensitivity: 0.8525266108569668, T2 Fear of bodily sensations: 0.9266197412328823, T2 Dysfunctional a

### 1.3 Class definitions prompt

In [74]:
def create_class_definitions_prompt(data_row):
    """
    Create a class definitions prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2."

    prompt = f"Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [88]:
# Test the function
data_row = X.iloc[4]
class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
print(class_definitions_prompt)
print("\n", target_names[0], y.iloc[4])

Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.0346394734930797, T1 Social support: -1.9443495599450165, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -1.2777276005686404, T1 Problem-focused coping: -0.6434380872019897, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: -1.6912121763530454, T1 Fear of bodily sensations: -1.3399218609334862, T1 Dysfunctional attitudes: 1.528535706802646, T1 General psychopathology: Global Severity Index (GSI): -0.5273858134379611, Education: 1.0, T1 BMI: 1.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: 1.0533586187543231, T2 General self-efficacy: 1.3559331135757966, T2 Life satisfaction: 1.005628346789385, T2 Anxiety sensitivity: -1.1666271173074825, T2 Fear of bodily sensations: -1.0416157340293282, T2 Dysfunctional attitudes: -0.8267579141727915, T2 General psychopathology: Global S

In [76]:
# Create a class definitions prompt for each row in the test set
X_test_class_definitions_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
    X_test_class_definitions_prompt.append(class_definitions_prompt)

X_test_class_definitions_prompt[0]

'Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9916025104337652, T1 Social support: 0.1912554943776842, T1 General self-efficacy: -1.209478876081861, T1 Life satisfaction: 0.1815598805259552, T1 Problem-focused coping: -1.0834346466693496, T1 Emotion-focused coping: -0.1263180001181833, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sensations: 1.2360549695511902, T1 Dysfunctional attitudes: -0.6885415900752596, T1 General psychopathology: Global Severity Index (GSI): -0.4917971179478549, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.7828728972526869, T2 Social support: 0.0984652458135236, T2 General self-efficacy: -1.1421997527016898, T2 Life satisfaction: -0.6556858132553557, T2 Anxiety sensitivity: 1.5262249302236135, T2 Fear of bodily sensations: 1.9574857842600144, T2 Dysfunctional attitudes: -0.2807869674967928, T2 General psychopathology: Global Se

### 1.4 Profiled simple prompt

In [77]:
def create_profiled_simple_prompt(data_row):
    """
    Create a profiled simple prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Tell me if the following data shows the development of a psychological disorder between time point T1 and T2. Just answer YES or NO."

    prompt = f"Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [89]:
# Test the function
data_row = X.iloc[4]
profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
print(profiled_simple_prompt)
print("\n", target_names[0], y.iloc[4])

Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.0346394734930797, T1 Social support: -1.9443495599450165, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -1.2777276005686404, T1 Problem-focused coping: -0.6434380872019897, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: -1.6912121763530454, T1 Fear of bodily sensations: -1.3399218609334862, T1 Dysfunctional attitudes: 1.528535706802646, T1 General psychopathology: Global Severity Index (GSI): -0.5273858134379611, Education: 1.0, T1 BMI: 1.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: 1.0533586187543231, T2 General self-efficacy: 1.3559331135757966, T2 Life satisfaction: 1.005628346789385, T2 Anxiety sensitivity: -1.1666271173074825, T2 Fear of bodily sensations: -1.0416157340293282, T2 Dysfunctional attitudes: -0.8267579141727915, T2 General psychopathology: Global S

In [79]:
# Create a profiled simple prompt for each row in the test set
X_test_profiled_simple_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
    X_test_profiled_simple_prompt.append(profiled_simple_prompt)

X_test_profiled_simple_prompt[0]

'Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9916025104337652, T1 Social support: 0.1912554943776842, T1 General self-efficacy: -1.209478876081861, T1 Life satisfaction: 0.1815598805259552, T1 Problem-focused coping: -1.0834346466693496, T1 Emotion-focused coping: -0.1263180001181833, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sensations: 1.2360549695511902, T1 Dysfunctional attitudes: -0.6885415900752596, T1 General psychopathology: Global Severity Index (GSI): -0.4917971179478549, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.7828728972526869, T2 Social support: 0.0984652458135236, T2 General self-efficacy: -1.1421997527016898, T2 Life satisfaction: -0.6556858132553557, T2 Anxiety sensitivity: 1.5262249302236135, T2 Fear of bodily sensations: 1.9574857842600144, T2 Dysfunctional attitudes: -0.2807869674967928, T2 General psychopathology: Global Se

### 1.5 Few-shot prompt

In [100]:
def create_few_shot_prompt(data_row, few_shot_examples):
    """
    Create a few-shot prompt for zero-shot classification.
    """

    example_data = []

    # create example prompt but including the target aswell, stating Example 1: ..., Example 2: ...
    for i, example in enumerate(few_shot_examples):
        example_data.append(
            f"Example {i + 1}: " + ", ".join([f"{name}: {value}" for name, value in zip(predictor_names + target_names, example)])
        )

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond with YES or NO."

    prompt1 = f"Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: {example_data}"

    prompt2 = f"Based on the previous example data prompts, classify the following data. Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    # concatenate
    prompt = f"{prompt1} {prompt2}"

    return instruction, prompt

In [175]:
def get_similar_examples(data_row, dat_train, n):
    """
    Get the top-n semantically similar examples to the test product from the training set.
    """

    # get columns that we want to use for similarity
    preds = dat_train.drop("hpi", axis = 1).columns
    dat_preds = dat_train[preds]
    data_row = data_row[preds]
    data_row = np.array([data_row])

    # dictionary to store the distances
    distances = {}

    for row in dat_preds.iterrows():
        # create array with the row and the data_row
        row_input = np.array([row[1]])
        dist_row = np.concatenate((data_row, row_input), axis = 0)

        # calculate the distance between the two rows
        dist = pdist(dist_row, metric = "euclidean")

        # store the distance in the dictionary
        distances[row[0]] = dist[0]

    # sort the dictionary by distance
    sorted_distances = sorted(distances.items(), key = lambda x: x[1])

    # get top keys
    keys = []
    for key in sorted_distances[:n]:
        keys.append(key[0])

    # drop all rows that are not in keys
    ex_dat = dat_train.loc[keys]

    return ex_dat

In [181]:
# Test the function
data_row = X.iloc[4]
few_shot_examples = close_ex.values
few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
print(few_shot_prompt)

Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: ['Example 1: T1 Positive mental health: -0.5131209919634224, T1 Social support: 0.6361732140282529, T1 General self-efficacy: 0.8110223827949994, T1 Life satisfaction: 0.1815598805259552, T1 Problem-focused coping: 0.1485557198392581, T1 Emotion-focused coping: -0.6121366420821499, T1 Anxiety sensitivity: 0.6872914724637128, T1 Fear of bodily sensations: -0.5766694667158019, T1 Dysfunctional attitudes: -0.9813631198515868, T1 General psychopathology: Global Severity Index (GSI): -0.278264945007215, Education: 1.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.7828728972526869, T2 Social support: 0.6714012695780124, T2 General self-efficacy: 0.9017271378889808, T2 Life satisfaction: -0.4939649658173775, T2 Anxiety sensitivity: -0.781933967660183, T2 Fear of bodily sensations: -0.9448705237619214, T2 Dys

In [183]:
# Create a few-shot prompt for each row in the test set
X_test_few_shot_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    few_shot_examples = get_similar_examples(data_row, dat_train, 5).values
    few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
    X_test_few_shot_prompt.append(few_shot_prompt)

X_test_few_shot_prompt[0]

"Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: ['Example 1: T1 Positive mental health: -0.5131209919634224, T1 Social support: -0.0756951374126611, T1 General self-efficacy: -0.0869781767058276, T1 Life satisfaction: 0.9922751478007364, T1 Problem-focused coping: -0.9074360228824055, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sensations: 1.6176811666600328, T1 Dysfunctional attitudes: -0.3957200602989325, T1 General psychopathology: Global Severity Index (GSI): 0.1487994008740618, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: -0.1880027660686982, T2 General self-efficacy: -0.915096764858282, T2 Life satisfaction: -0.8174066606933537, T2 Anxiety sensitivity: 2.1673801796357792, T2 Fear of bodily sensations: 2.344466625329608, T2 Dy

### 1.6 Vignette text prompt

In [184]:
def interpret_value(val, thresh_low = -0.25, thresh_high = 0.25):
    """Convert standardized value to qualitative description with value."""
    val_str = f"{val:.3f}"
    if val > thresh_high:
        return f"above average ({val_str})"
    elif val < thresh_low:
        return f"below average ({val_str})"
    else:
        return f"average ({val_str})"

In [193]:
def create_vignette_prompt(data):

    # demographics
    bmi = data['bmi_kat']
    education = data['bild']
    ses = data['ses_kom']

    # time point 1
    t1_pm_health = interpret_value(data['whi_ges'])
    t1_support = interpret_value(data['soz_ges'])
    t1_self_eff = interpret_value(data['gke_ges'])
    t1_life_sat = interpret_value(data['lzh_ges'])
    t1_coping_prob = interpret_value(data['woc_gesp'])
    t1_coping_emot = interpret_value(data['woc_gese'])
    t1_anx_sens = interpret_value(data['asi_ges'])
    t1_fear = interpret_value(data['bsq_ges'])
    t1_dysfunc = interpret_value(data['das_ges'])
    t1_gsi = interpret_value(data['scl_gsi'])
    t1_stress = interpret_value(data['ile'])

    # time point 2
    t2_pm_health = interpret_value(data['whi_gesy'])
    t2_support = interpret_value(data['soz_gesy'])
    t2_self_eff = interpret_value(data['gke_gesy'])
    t2_life_sat = interpret_value(data['lzh_gesy'])
    t2_anx_sens = interpret_value(data['asi_gesy'])
    t2_fear = interpret_value(data['bsq_gesy'])
    t2_dysfunc = interpret_value(data['das_gesy'])
    t2_gsi = interpret_value(data['scl_gsiy'])

    # generate prompt
    vignette = f"""A woman with a BMI of {bmi}, an education level of {education}, and a socioeconomic status of {ses} has the following psychological profile: At Time Point 1, she showed {t1_pm_health} positive mental health, {t1_support} social support, and {t1_self_eff} general self-efficacy. Her life satisfaction was {t1_life_sat}, and she relied on {t1_coping_prob} problem-focused and {t1_coping_emot} emotion-focused coping strategies. Anxiety sensitivity was {t1_anx_sens}, and her fear of bodily sensations was {t1_fear}. She reported {t1_dysfunc} levels of dysfunctional attitudes and {t1_gsi} levels of general psychopathology. Her stress level was {t1_stress}. By Time Point 2, approximately 17 months later, she reported {t2_pm_health} positive mental health, {t2_support} social support, and {t2_self_eff} self-efficacy. Life satisfaction remained {t2_life_sat}. However, anxiety sensitivity had become {t2_anx_sens}, and fear of bodily sensations was {t2_fear}. Dysfunctional attitudes were {t2_dysfunc}, and general psychopathology was {t2_gsi}. Does this person develop a psychological disorder between time point T1 and T2?"""
    return vignette

In [194]:
# Test the function
data_row = X.iloc[4]
vignette_prompt = create_vignette_prompt(data_row)
print(vignette_prompt)

A woman with a BMI of 1.0, an education level of 1.0, and a socioeconomic status of 2.0 has the following psychological profile: At Time Point 1, she showed average (-0.035) positive mental health, below average (-1.944) social support, and above average (0.587) general self-efficacy. Her life satisfaction was below average (-1.278), and she relied on below average (-0.643) problem-focused and below average (-0.450) emotion-focused coping strategies. Anxiety sensitivity was below average (-1.691), and her fear of bodily sensations was below average (-1.340). She reported above average (1.529) levels of dysfunctional attitudes and below average (-0.527) levels of general psychopathology. Her stress level was below average (-0.363). By Time Point 2, she now reported below average (-0.275) positive mental health, above average (1.053) social support, and above average (1.356) self-efficacy. Life satisfaction remained above average (1.006). However, anxiety sensitivity had become below ave

In [195]:
# Create a vignette prompt for each row in the test set
X_test_vignette_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    vignette_prompt = create_vignette_prompt(data_row)
    X_test_vignette_prompt.append(vignette_prompt)

X_test_vignette_prompt[0]

'A woman with a BMI of 2.0, an education level of 2.0, and a socioeconomic status of 2.0 has the following psychological profile: At Time Point 1, she showed below average (-0.992) positive mental health, average (0.191) social support, and below average (-1.209) general self-efficacy. Her life satisfaction was average (0.182), and she relied on below average (-1.083) problem-focused and average (-0.126) emotion-focused coping strategies. Anxiety sensitivity was above average (1.084), and her fear of bodily sensations was above average (1.236). She reported below average (-0.689) levels of dysfunctional attitudes and below average (-0.492) levels of general psychopathology. Her stress level was below average (-0.763). By Time Point 2, she now reported below average (-0.783) positive mental health, average (0.098) social support, and below average (-1.142) self-efficacy. Life satisfaction remained below average (-0.656). However, anxiety sensitivity had become above average (1.526), and